In [2]:
import os, sys, wave, struct

import numpy as np
import pyaudio
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import scipy.ndimage.filters

from copy import deepcopy
from math import ceil
import time

from scipy.io.wavfile import write

In [3]:
def upsampling(x, L):
    zero = np.zeros(len(x)*L)
    zero[::L] = x
    return zero

def downsampling(x, M):
    return x[::M]*M

def delay(x, z):
    zero=np.zeros(len(x))
    if(z<0):
        zero[:len(x) - z*(-1):]=x[z*(-1)::]
    else:
        zero[z::]=x[:len(x) - z:]
    return zero

def load_sound(file):
    return wave.open(file, 'rb')

def play_sound(file, chunk = 1024):
    """
    Script from PyAudio doc
    """
    wf = load_sound(file)
    p = pyaudio.PyAudio()
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)
    data = wf.readframes(chunk)

    while data:
        stream.write(data)
        data = wf.readframes(chunk )

    stream.stop_stream()
    stream.close()
    p.terminate()

def getAudio(filepath):
    data_path = os.getcwd()
    sound = os.path.join(data_path, filepath)
    wavefile = wave.open(sound, 'rb')
    play = False
    if play :
        play_sound(sound) 
    Fs = int(wavefile.getframerate())
    num_samples = int(wavefile.getnframes())
    data = wavefile.readframes(num_samples)
    data = struct.unpack('{n}h'.format(n=num_samples), data)
    return np.array(data)
def F0 (x):
    return delay(x, -1)
def F1 (x):
    return x

## Exercise 1
### 1.1

3

## 2

In [78]:
s0 = getAudio('femme3.wav')
s1 = getAudio('homme3.wav')

In [79]:
snh0 = upsampling(s0, 2)
snh1 = upsampling(s1, 2)

In [80]:
temp = F0(snh0) + F1(snh1)

In [81]:
sch0 = F1(temp)
sch1 = F0(temp)

In [82]:
s0new = downsampling(sch0, 2)
s1new = downsampling(sch1, 2)

In [83]:
write('femme2_1.wav',22000,np.array(s0new*0.4,dtype=np.int16))
write('homme2_1.wav',22000,np.array(s1new*0.4,dtype=np.int16))

### 2.2

They are not the right outputs because they were swapped. In the expected audio file for the female voice the male audio file was present. And the other way around as well.

### 2.3

In [31]:
def C(x, a):
    aux = np.zeros(2)
    aux[0]=1
    aux[1]=-a
    return np.convolve(x, aux)

In [67]:
def Dcausal(x, a):
    #n0 equals zero, d equals 1
    aux =np.ones(100)
    for i in range(100):
        aux[i]=a**i
    return np.convolve(x, aux)

In [68]:
def DantiCausal(x, a):
    aux = np.ones(100)
    n0 = 102
    for i in range(100):
        aux[i]=-a**(i-n0)*unit(-i+n0-1)
    return np.convolve(x, aux)

2.3 a

In [76]:
femme2_3 = Dcausal(C(temp, 0.9), 0.9)
femme2_3 = F0(femme2_3)
femme2_3 = downsampling(femme2_3, 2)
write('femme2_3a.wav',22000,np.array(femme2_3*0.5,dtype=np.int16))

homme2_3 = Dcausal(C(temp, 0.9), 0.9)
homme2_3 = F1(homme2_3)
homme2_3 = downsampling(homme2_3, 2)
write('homme2_3a.wav',22000,np.array(homme2_3*0.4,dtype=np.int16))

2.3 b

In [70]:
def unit(x):
    if x >=0:
        return 1
    else:
        return 0

In [75]:
femme2_3 = DantiCausal(C(temp, 1.2), 1.2)
femme2_3 = F0(femme2_3)
femme2_3 = downsampling(femme2_3, 2)
write('femme2_3b.wav',22000,np.array(femme2_3*0.5,dtype=np.int16))

homme2_3 = DantiCausal(C(temp, 1.2), 1.2)
homme2_3 = F1(homme2_3)
homme2_3 = downsampling(homme2_3, 2)
write('homme2_3b.wav',22000,np.array(homme2_3*0.5,dtype=np.int16))